In [1]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, classification_report

## Defining the bucket 
s3 = boto3.resource('s3')
bucket_name = 'data-448'
bucket = s3.Bucket(bucket_name)

## Defining the csv file 
file_key = 'In_Class_Assignments/telecom_churn.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
churn_data = pd.read_csv(file_content_stream)
churn_data.head()

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.7,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.7,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.0,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.0,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.0,3,166.7,113,41.0,7.42,10.1


In [2]:
## Relative frequency table
churn_data['Churn'].value_counts() / churn_data.shape[0]

0    0.855086
1    0.144914
Name: Churn, dtype: float64

In [3]:
## Defining the input and target variables
X = churn_data[['AccountWeeks', 'ContractRenewal', 'CustServCalls', 'MonthlyCharge', 'DayMins']]
Y = churn_data['Churn']

## Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [4]:
## Random forest model 
RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

## Predicting on the test dataset
RF_pred = RF_md.predict_proba(X_test)[:, 1]

## Computing the ROC curve
fpr, tpr, threshold = roc_curve(Y_test, RF_pred)

## Creating a data-frame 
cutoffs = pd.DataFrame({'False_Positive': fpr, 'True_Positive': tpr, 'cutoff': threshold})

## Finding the optimal cut-off
cutoffs['True_Positive_minus_1'] = cutoffs['True_Positive'] - 1
cutoffs['Euclidean_dist'] = np.sqrt(cutoffs['False_Positive']**2 + cutoffs['True_Positive_minus_1']**2)

## Sorting based on the Euclidean distance
cutoffs = cutoffs.sort_values(by = 'Euclidean_dist').reset_index(drop = True)

## Changing likelihoods to labels
RF_pred = np.where(RF_pred < cutoffs['cutoff'][0], 0, 1)

## Printing classification report 
print(classification_report(Y_test, RF_pred))

              precision    recall  f1-score   support

           0       0.97      0.84      0.90       570
           1       0.47      0.85      0.60        97

    accuracy                           0.84       667
   macro avg       0.72      0.84      0.75       667
weighted avg       0.90      0.84      0.86       667



In [8]:
## Adaboost model 
Ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = 0.01).fit(X_train, Y_train)

## Predicting on the test dataset
Ada_pred = Ada_md.predict_proba(X_test)[:, 1]

## Computing the ROC curve
fpr, tpr, threshold = roc_curve(Y_test, Ada_pred)

## Creating a data-frame 
cutoffs = pd.DataFrame({'False_Positive': fpr, 'True_Positive': tpr, 'cutoff': threshold})

## Finding the optimal cut-off
cutoffs['True_Positive_minus_1'] = cutoffs['True_Positive'] - 1
cutoffs['Euclidean_dist'] = np.sqrt(cutoffs['False_Positive']**2 + cutoffs['True_Positive_minus_1']**2)

## Sorting based on the Euclidean distance
cutoffs = cutoffs.sort_values(by = 'Euclidean_dist').reset_index(drop = True)

## Changing likelihoods to labels
Ada_pred = np.where(Ada_pred < cutoffs['cutoff'][0], 0, 1)

## Printing classification report 
print(classification_report(Y_test, Ada_pred))

              precision    recall  f1-score   support

           0       0.97      0.86      0.91       570
           1       0.52      0.86      0.64        97

    accuracy                           0.86       667
   macro avg       0.74      0.86      0.78       667
weighted avg       0.91      0.86      0.88       667

